In [37]:
import numpy as np
import pandas as pd

In [38]:
pd.options.display.max_colwidth = 500

In [39]:
YEARS = range(2007, 2017)

In [40]:
fin_dictionary_df = pd.read_csv('../references/financial_support_dictionary_datasd.csv')
fin_recipients_dictionary_df = pd.read_csv('../references/financial_support_recipients_dictionary_datasd.csv')

In [41]:
fin_dictionary_df[['Column name', 'Description']]

,Column name,Description
0,form,Number of the state form used to report the transaction
1,schedule,Letter of the schedule used to report the transaction
2,schedule_description,A short description of the form used to report the transaction
3,recipient_ID,A unique identifier issued to election committees
4,recipient_name,Name of the election committee receiving support or the candidate or ballot measure on whose behalf money is spent
5,filing_date,Date the form used to report the transaction was filed
6,report_period_from,The beginning period of time covered by the form used to report the transaction
7,report_period_to,The end period of time covered by the form used to report the transaction
8,contributor_code,A code identifying the type of contributor
9,contributor_last,"The primary name of the contributor. This could be a last name, a full name or a business name"


In [42]:
fin_recipients_dictionary_df

,column_name,description
0,reported_name,"Name of recipient as it appears in the recipient_name column of the transactions data. Due to human error in reporting campaign finance transactions, a misspelling or inconsistency would cause a recipient to be listed multiple times for the same year."
1,year_reported,"The year during which this candidate received financial support. Any one candidate may have received support during multiple years, and even be listed multiple times for the same year if the name was inconsistent or misspelled. The candidate is listed for every variation of name that appears within a year. The year corresponds to the year in the filenames of the transactions data."
2,candidate_or_measure,"If the recipient is a candidate or measure on whose behalf money was spent, or a committee with a purpose to support a specific candidate or measure, the candidate or measure is here."
3,corrected_name,"The reported name, corrected for any misspellings or inconsistencies."
4,committee_id,"Committee ID number, if recipient is a committee. ID numbers can be used to find committees in campaign finance databases. Depending on the committee, filings may appear in city, county or state data."
5,candidate_office_sought,"If the recipient is a candidate on whose behalf money was spent, or a committee with a purpose to support a specific candidate, the office sought by this candidate is here."
6,support_or_oppose,"If the recipient is a committee focused on a specific candidate or ballot measure, this column lists whether the committee supports or opposes the candidate."


In [43]:
fin_recipients_df = pd.read_csv('../data/raw/financial_support_recipients_datasd.csv')

In [44]:
fin_recipients_df.head()

,recipient_name,year_reported,candidate_or_measure,corrected_name,committee_id,candidate_office_sought,support_or_oppose
0,A Lot of People Who Support Stephen Whitburn,2007,Stephen Whitburn,A Lot of People Who Support Stephen Whitburn,1298544,San Diego City Council District 3,Support
1,April Boling for City Council,2007,April Boling,April Boling for City Council,1298534,San Diego City Council District 7,Support
2,Brian Maienschein for City Council,2007,Brian Maienschein,Brian Maienschein for City Council,1235908,San Diego City Council District 5,Support
3,Carl DeMaio for City Council 2012,2007,Carl DeMaio,Carl DeMaio for City Council 2012,1297980,San Diego City Council District 5,Support
4,Committee to Elect Marshall Merrifield,2007,Marshall Merrifield,Committee to Elect Marshall Merrifield,1298864,San Diego City Council District 1,Support


In [45]:
len(fin_recipients_df['corrected_name'].unique())

613

In [46]:
# Couldn't figure out a good way to clean this up :(
fin_recipients_df = fin_recipients_df.drop_duplicates(subset=('year_reported', 'recipient_name'))

In [47]:
for year in YEARS:
    fin_recipients_year_df = fin_recipients_df[fin_recipients_df['year_reported'] == year]
    duplicates = fin_recipients_year_df[fin_recipients_year_df.duplicated(subset='recipient_name')]
    if len(duplicates) > 0:
        print("Found duplicates in {year}".format(year=year))
        print(duplicates)

In [48]:
fin_support_df = pd.DataFrame()
for year in YEARS:
    filename = "../data/raw/financial_support_{year}_datasd.csv".format(year=year)
    fin_support_year_df = pd.read_csv(filename)
    fin_recipients_year_df = fin_recipients_df[fin_recipients_df['year_reported'] == year]
    fin_support_year_full_df = pd.merge(fin_support_year_df, fin_recipients_year_df, on='recipient_name')
    fin_support_df = fin_support_df.append(fin_support_year_full_df, ignore_index=True)

In [49]:
len(fin_support_df)

169484

In [50]:
total_rows = 0
for year in YEARS:
    filename = "../data/raw/financial_support_{year}_datasd.csv".format(year=year)
    total_rows += len(pd.read_csv(filename))
total_rows

169486

In [51]:
fin_support_df.head()

,form,schedule,schedule_description,recipient_id,recipient_name,report_period_from,report_period_to,contributor_code,contributor_last,contributor_first,...,intermediary_emp,intermediary_occ,filing_id,report_year,year_reported,candidate_or_measure,corrected_name,committee_id,candidate_office_sought,support_or_oppose
0,460,A,Monetary contributions,1298534,April Boling for City Council,2007-01-01T00:00:00.0000000-08:00,2007-06-30T00:00:00.0000000-07:00,IND,Traylor,Robert C.,...,NaN,NaN,6822055,2007,2007,April Boling,April Boling for City Council,1298534,San Diego City Council District 7,Support
1,460,A,Monetary contributions,1298534,April Boling for City Council,2007-01-01T00:00:00.0000000-08:00,2007-06-30T00:00:00.0000000-07:00,IND,Cloward,Richard S.,...,NaN,NaN,6822055,2007,2007,April Boling,April Boling for City Council,1298534,San Diego City Council District 7,Support
2,460,A,Monetary contributions,1298534,April Boling for City Council,2007-01-01T00:00:00.0000000-08:00,2007-06-30T00:00:00.0000000-07:00,IND,Buhler,Luis P.,...,NaN,NaN,6822055,2007,2007,April Boling,April Boling for City Council,1298534,San Diego City Council District 7,Support
3,460,A,Monetary contributions,1298534,April Boling for City Council,2007-01-01T00:00:00.0000000-08:00,2007-06-30T00:00:00.0000000-07:00,IND,Sutton,James R.,...,NaN,NaN,6822055,2007,2007,April Boling,April Boling for City Council,1298534,San Diego City Council District 7,Support
4,460,A,Monetary contributions,1298534,April Boling for City Council,2007-01-01T00:00:00.0000000-08:00,2007-06-30T00:00:00.0000000-07:00,IND,Wahl,A. Christopher,...,NaN,NaN,6822055,2007,2007,April Boling,April Boling for City Council,1298534,San Diego City Council District 7,Support


In [52]:
fin_support_df.columns

Index([u'form', u'schedule', u'schedule_description', u'recipient_id',
       u'recipient_name', u'report_period_from', u'report_period_to',
       u'contributor_code', u'contributor_last', u'contributor_first',
       u'contributor_city', u'contributor_state', u'contributor_zip',
       u'contributor_emp', u'contributor_occ', u'contribution_date',
       u'contribution_amount', u'contribution_annual', u'contribution_desc',
       u'contributor_id', u'intermediary_last', u'intermediary_first',
       u'intermediary_city', u'intermediary_state', u'intermediary_zip',
       u'intermediary_emp', u'intermediary_occ', u'filing_id', u'report_year',
       u'year_reported', u'candidate_or_measure', u'corrected_name',
       u'committee_id', u'candidate_office_sought', u'support_or_oppose'],
      dtype='object')

In [53]:
fin_support_positive_amounts = fin_support_df[fin_support_df['contribution_amount'] > 0]

In [54]:
fin_support_relevant_columns = fin_support_positive_amounts[['corrected_name',
                                                             'contribution_amount',
                                                             'contributor_first',
                                                             'contributor_last',
                                                             'report_year',
                                                             'candidate_or_measure',
                                                             'candidate_office_sought',
                                                             'support_or_oppose']]

In [55]:
fin_support_no_null = fin_support_relevant_columns.fillna('')

In [56]:
def contributor_name(row):
    first = row['contributor_first']
    last = row['contributor_last']
    return ' '.join([first, last])
    

fin_support_with_name = fin_support_no_null.assign(name=fin_support_no_null.apply(contributor_name, axis='columns'))

In [57]:
fin_support_with_name

,corrected_name,contribution_amount,contributor_first,contributor_last,report_year,candidate_or_measure,candidate_office_sought,support_or_oppose,name
0,April Boling for City Council,100.00,Robert C.,Traylor,2007,April Boling,San Diego City Council District 7,Support,Robert C. Traylor
1,April Boling for City Council,100.00,Richard S.,Cloward,2007,April Boling,San Diego City Council District 7,Support,Richard S. Cloward
2,April Boling for City Council,100.00,Luis P.,Buhler,2007,April Boling,San Diego City Council District 7,Support,Luis P. Buhler
3,April Boling for City Council,270.00,James R.,Sutton,2007,April Boling,San Diego City Council District 7,Support,James R. Sutton
4,April Boling for City Council,270.00,A. Christopher,Wahl,2007,April Boling,San Diego City Council District 7,Support,A. Christopher Wahl
5,April Boling for City Council,100.00,James L.,Abbott,2007,April Boling,San Diego City Council District 7,Support,James L. Abbott
6,April Boling for City Council,100.00,Lin J.,Ball,2007,April Boling,San Diego City Council District 7,Support,Lin J. Ball
7,April Boling for City Council,270.00,David I.,Cohn,2007,April Boling,San Diego City Council District 7,Support,David I. Cohn
8,April Boling for City Council,250.00,Abbe S.,Wolfsheimer,2007,April Boling,San Diego City Council District 7,Support,Abbe S. Wolfsheimer
9,April Boling for City Council,200.00,Carol,Reed,2007,April Boling,San Diego City Council District 7,Support,Carol Reed
